In [ ]:
""" import settings """
%load_ext autoreload
%autoreload 2

import numpy as np
import sys, os, csv

from utils_invitro_global import *

from utils_invitro_misc import parse_Hf_filename

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib_settings import set_plot_settings, reset_plot_settings

set_plot_settings()

In [ ]:
"""
Gain was characterized by injecting 1kHz sinusoid, and taking recordings in both 
16x16 and 32x32 modes. They were programmed in the most dense tiles, and were moved
around the array to obtain full coverage.

Chip Configurations Used:
ibias global 4, ibias rec 4, vga_gain 0, vbias_pr 7

Noise characterization was performed in the same way, except that instead of sinusoid input,
the MEA and Counter wells were shorted through Ag/AgCl. Integration bandwidth was 10 to 4kHz
No filtering was applied - this means that 60 Hz harmonics may be included in the integration.

Pixels whose gain is below a threshold (300 V/V) have been marked as invalid (nan).
Reasons for low gain include packaging, poor electrode impedance, DC offset, etc.
"""

In [ ]:
""" Load Data """
load_dir  = f'{REC_DATA_DIR}/array_map'

gain_map_128x128  = np.load(f'{load_dir}/gain_map_128x128.npy')
gain_map_256x256  = np.load(f'{load_dir}/gain_map_256x256.npy')

# noise_map_128x128 and noise_map_256x256 are integrated OUTPUT noise
noise_map_128x128 = np.load(f'{load_dir}/noise_map_128x128.npy')
noise_map_256x256 = np.load(f'{load_dir}/noise_map_256x256.npy')

In [ ]:
""" print pixel yield stats """
assert np.count_nonzero(np.isnan(gain_map_128x128)) == np.count_nonzero(np.isnan(noise_map_128x128))
assert np.count_nonzero(np.isnan(gain_map_256x256)) == np.count_nonzero(np.isnan(noise_map_256x256))

count_excl_128x128 = np.count_nonzero(np.isnan(noise_map_128x128))
count_excl_256x256 = np.count_nonzero(np.isnan(noise_map_256x256))

count_incl_128x128 = 128*128 - count_excl_128x128
count_incl_256x256 = 256*256 - count_excl_256x256

print(f'excluded: {count_excl_128x128}, {count_excl_256x256}')
print(f'included: {count_incl_128x128}, {count_incl_256x256}')
print(f'yield: {count_incl_128x128/(128*128)*100:.2f}, {count_incl_256x256/(256*256)*100:.2f}')

Plot Gain Histogram

In [ ]:
""" Convert Gain from V/V to dB """
gain_map_db_128x128 = 20*np.log10(gain_map_128x128)
gain_map_db_256x256 = 20*np.log10(gain_map_256x256)

# find mean and std dev.
gain_mu    = np.nanmean(gain_map_db_128x128)
gain_sigma = np.nanstd(gain_map_db_128x128)
tetrode_gain_mu    = np.nanmean(gain_map_db_256x256)
tetrode_gain_sigma = np.nanstd(gain_map_db_256x256)

In [ ]:
""" Plot Gain Histogram """
fig, ax = plt.subplots(figsize=(4, 3))

# Generate Histogram data
vmin = min(np.nanmin(gain_map_128x128), np.nanmin(gain_map_256x256))
vmax = max(np.nanmax(gain_map_128x128), np.nanmax(gain_map_256x256))
db_vmin, db_vmax = 20*np.log10(vmin), 20*np.log10(vmax)

frq, edges = np.histogram(gain_map_db_128x128, bins=50, range=(db_vmin, db_vmax))
tetrode_frq, tetrode_edges = np.histogram(gain_map_db_256x256, bins=50, range=(db_vmin, db_vmax))

# Plot Histogram
ax.bar(tetrode_edges[:-1], tetrode_frq, width=np.diff(edges), edgecolor='k', align="edge")
ax.bar(edges[:-1], frq, width=np.diff(edges), edgecolor='k', align="edge")
ax.set_xlim((51, 54.5))

ax.set_xticks([51, 52, 53, 54])
ax.set_yticks([1000, 3000, 5000])

# Add Labels
ax.set_xlabel('Gain (dB)')
ax.set_ylabel('Count')

title_str = f'Non-tet: {gain_mu:.2f} ± {gain_sigma:.2f} dB (SD) \n'
title_str += f'tetrode: {tetrode_gain_mu:.2f} ± {tetrode_gain_sigma:.2f} dB (SD) \n'
print(title_str)
ax.legend(['32×32', '16×16'], fontsize=12, loc=(0.6, 0.7))

In [ ]:
"""
Since gain is programmed through the back-end amplifier which is shared by all pixels,
overall system gain variation is equivalent to the pixel gain variation and is independent
of the PGA configuration
"""

In [ ]:
""" normalize """
norm_gain_128x128 = gain_map_128x128/np.nanmean(gain_map_128x128)
norm_gain_256x256 = gain_map_256x256/np.nanmean(gain_map_256x256)

# mean, std, min, max
norm_mu, norm_sigma = np.nanmean(norm_gain_128x128), np.nanstd(norm_gain_128x128)
tetrode_norm_mu, tetrode_norm_sigma = np.nanmean(norm_gain_256x256), np.nanstd(norm_gain_256x256)
norm_vmin = min(np.nanmin(norm_gain_128x128), np.nanmin(norm_gain_256x256))
norm_vmax = max(np.nanmax(norm_gain_128x128), np.nanmax(norm_gain_256x256))

In [ ]:
""" Plot Normalized Gain Histogram """
fig, ax = plt.subplots(figsize=(4, 3))

# Generate Histogram
frq, edges = np.histogram(norm_gain_128x128, bins=50, range=(norm_vmin, norm_vmax))
tetrode_frq, tetrode_edges = np.histogram(norm_gain_256x256, bins=50, range=(norm_vmin, norm_vmax))

# Plot Histogram
ax.bar(tetrode_edges[:-1], tetrode_frq, width=np.diff(edges), edgecolor='k', align="edge")
ax.bar(edges[:-1], frq, width=np.diff(edges), edgecolor='k', align="edge")
ax.set_xlim((0.75, 1.25))

# Add Labels
ax.set_xlabel('Normalized Gain')
ax.set_ylabel('Count')

title_str = f'Non-tet: {norm_mu:.2f} ± {norm_sigma:.3f} (SD) \n'
title_str += f'tetrode: {tetrode_norm_mu:.2f} ± {tetrode_norm_sigma:.3f} (SD) \n'
ax.legend(['32×32', '16×16'], fontsize=12, loc=(0.6, 0.7))
print(title_str)

Plot Noise Histogram

In [ ]:
""" convert output noise to input-referred microvolt_rms """
irn_128x128 = noise_map_128x128/gain_map_128x128/1e-6
irn_256x256 = noise_map_256x256/gain_map_256x256/1e-6

In [ ]:
""" Print IRN Stats"""
noise_mu    = np.nanmean(irn_128x128)
noise_sigma = np.nanstd(irn_128x128)
tetrode_noise_mu    = np.nanmean(irn_256x256)
tetrode_noise_sigma = np.nanstd(irn_256x256)

print(f'Mean: {noise_mu:.2f}. Std Dev: {noise_sigma:.2f}')
print(f'Mean: {tetrode_noise_mu:.2f}. Std Dev: {tetrode_noise_sigma:.2f}')

In [ ]:
""" Plot IRN Histogram """
fig, ax = plt.subplots(figsize=(4, 3))

# Generate Histogram
irn_vmin, irn_vmax = np.nanmin(irn_128x128), 35
frq, edges = np.histogram(irn_128x128, bins=50, range=(irn_vmin, irn_vmax))
tetrode_frq, tetrode_edges = np.histogram(irn_256x256, bins=50, range=(irn_vmin, irn_vmax))

# Plot Histogram
ax.bar(tetrode_edges[:-1], tetrode_frq, width=np.diff(edges), edgecolor='k', align="edge")
ax.bar(edges[:-1], frq, width=np.diff(edges), edgecolor='k', align="edge")
# ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.set_yticks([1000, 3000, 5000])
ax.set_xticks([5, 15, 25, 35])

# Add Labels
ax.set_xlabel(r'Input-referred noise (μV$_\text{RMS}$)', fontsize=16) 
ax.set_ylabel('Count')
ax.legend(['32×32', '16×16'], fontsize=12)

Plot Gain Maps

NaN values near the edges of the array are affected by the silicone flowing over them during the packaging process.

In [ ]:
""" Non-Tetrode Mode """
fig, ax = plt.subplots(figsize=(4, 4))

# Plot Map
img = ax.imshow(gain_map_db_128x128, vmin=db_vmin, vmax=db_vmax)
ax.set_xticks([])
ax.set_yticks([])

# Add a color bar to the plot
cx0 = ax.get_position().x0
cx1 = ax.get_position().x1
cy0 = ax.get_position().y0
cy1 = ax.get_position().y1
cbar_ax = fig.add_axes([cx1 + 0.03*2, cy0, 0.03, cy1 - cy0]) # [left, bottom, width, height]
cbar = fig.colorbar(img, cax=cbar_ax)
cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
cbar.set_ticks([db_vmin, (db_vmin + db_vmax)/2, db_vmax])
cbar.set_label('Gain (dB)') 

In [ ]:
""" Non-Tetrode Mode """
fig, ax = plt.subplots(figsize=(4, 4))

# Plot Map
img = ax.imshow(gain_map_db_256x256, vmin=db_vmin, vmax=db_vmax)
ax.set_xticks([])
ax.set_yticks([])

# Add a color bar to the plot
cx0 = ax.get_position().x0
cx1 = ax.get_position().x1
cy0 = ax.get_position().y0
cy1 = ax.get_position().y1
cbar_ax = fig.add_axes([cx1 + 0.03*2, cy0, 0.03, cy1 - cy0]) # [left, bottom, width, height]
cbar = fig.colorbar(img, cax=cbar_ax)
cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
cbar.set_ticks([db_vmin, (db_vmin + db_vmax)/2, db_vmax])
cbar.set_label('Gain (dB)') 

Plot Noise Maps

In [ ]:
""" Non-Tetrode Mode """
# Hot pixels near the top right corner are probably covered under a thin layer of silicone..
# So the IRN number reported from our characterization is probably worse than our chip's true IRN
fig, ax = plt.subplots(figsize=(4, 4))

# Plot Map
img = ax.imshow(irn_128x128, vmin=irn_vmin, vmax=irn_vmax)
ax.set_xticks([])
ax.set_yticks([])

# Add a color bar to the plot
cx0 = ax.get_position().x0
cx1 = ax.get_position().x1
cy0 = ax.get_position().y0
cy1 = ax.get_position().y1
cbar_ax = fig.add_axes([cx1 + 0.03*2, cy0, 0.03, cy1 - cy0]) # [left, bottom, width, height]
cbar = fig.colorbar(img, cax=cbar_ax)
cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
cbar.set_ticks([irn_vmin, (irn_vmin + irn_vmax)/2, irn_vmax])
cbar.set_label(r'Input-referred noise (μV$_\text{RMS}$)')

In [ ]:
""" Tetrode Mode """
# Hot pixels near the top right corner are probably covered under a thin layer of silicone..
# So the IRN number reported from our characterization is probably worse than our chip's true IRN
fig, ax = plt.subplots(figsize=(4, 4))

# Plot Map
irn_tetrode_vmin =np.nanmin(irn_256x256)
img = ax.imshow(irn_256x256, vmin=irn_tetrode_vmin, vmax=irn_vmax)
ax.set_xticks([])
ax.set_yticks([])

# Add a color bar to the plot
cx0 = ax.get_position().x0
cx1 = ax.get_position().x1
cy0 = ax.get_position().y0
cy1 = ax.get_position().y1
cbar_ax = fig.add_axes([cx1 + 0.03*2, cy0, 0.03, cy1 - cy0]) # [left, bottom, width, height]
cbar = fig.colorbar(img, cax=cbar_ax)
cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
cbar.set_ticks([irn_tetrode_vmin, (irn_tetrode_vmin + irn_vmax)/2, irn_vmax])
cbar.set_label(r'Input-referred noise (μV$_\text{RMS}$)')